# Calculate normalised enrichment scores of Bindea gene sets

Script related to figure 3a, 3b, s3c 

### Aim:
- Calculate NES of Bindea Gene Sets

In [1]:
sessionInfo()

R version 3.5.0 (2018-04-23)
Platform: x86_64-apple-darwin15.6.0 (64-bit)
Running under: macOS High Sierra 10.13.6

Matrix products: default
BLAS: /Library/Frameworks/R.framework/Versions/3.5/Resources/lib/libRblas.0.dylib
LAPACK: /Library/Frameworks/R.framework/Versions/3.5/Resources/lib/libRlapack.dylib

locale:
[1] C/UTF-8/C/C/C/C

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

loaded via a namespace (and not attached):
 [1] compiler_3.5.0  IRdisplay_0.6.1 pbdZMQ_0.3-3    tools_3.5.0    
 [5] htmltools_0.3.6 base64enc_0.1-3 crayon_1.3.4    Rcpp_1.0.1     
 [9] uuid_0.1-2      IRkernel_0.8.14 jsonlite_1.6    digest_0.6.18  
[13] repr_0.17       evaluate_0.13  

## Load packages

In [1]:
library(GSVA)
library(BBmisc)
library(nortest)

Warning message:
“package ‘GSVA’ was built under R version 3.5.1”
Attaching package: ‘BBmisc’

The following object is masked from ‘package:base’:

    isFALSE



In [2]:
setwd("~/git_repos/HGSOC_TME_Heterogeneity/Scripts/3/")

## Load expression data

In [2]:
mix <- "../../Data/1/TreatmentNaive_log2exp_loess_norm.txt"

In [3]:
data <- read.csv(mix,
                 sep='\t',
                 header=T,
                 row.names='Hugo_Symbol')

In [4]:
head(data)

,A01,A02,A03,A04,A05,A06,A07,A08,A09,A10,⋯,C06,C07,C08,C09,C10,C11,C12,D01,D02,D03
A1BG,4.124711,3.880633,4.864973,4.167882,4.317845,3.679057,4.092837,4.370571,4.252637,4.174694,⋯,3.804537,4.416981,4.169014,4.503313,4.146333,4.078398,4.011369,3.845128,3.600927,4.521857
A2BP1,3.960924,4.033715,3.721251,3.916073,4.338194,3.357396,3.872447,3.803388,3.484735,4.427405,⋯,4.991068,3.799913,4.117883,4.122134,4.221760,3.648326,3.675317,3.546925,3.788279,4.920841
A2M,12.603155,10.859838,12.612923,9.176999,10.548790,11.840776,7.200547,11.835658,11.535547,7.080118,⋯,9.061598,9.189459,8.844728,7.464440,6.726233,9.265911,10.740486,9.735582,10.781599,8.734304
A2ML1,3.945877,4.000621,3.767932,3.789729,3.543204,3.805520,4.336280,4.031757,4.166018,3.673146,⋯,4.231174,3.871651,3.287232,3.721093,3.723801,3.884499,3.694111,3.937682,3.600471,3.402861
A4GALT,3.597277,3.863757,4.051844,4.113169,3.975144,3.991293,5.439960,4.696177,4.171788,4.041633,⋯,4.183089,4.699729,4.019537,3.739865,4.591953,4.997860,4.099630,5.001239,4.714710,4.145119
A4GNT,3.714481,3.593169,3.241352,3.515850,3.364447,3.767865,3.768695,3.163977,3.423377,3.205154,⋯,3.896812,3.915653,3.351422,3.719759,3.641028,3.114521,3.560199,3.539738,3.760903,3.479266


In [5]:
data_mtx <- as.matrix(data)

## Load Bindea gene sets

In [7]:
Bindea_GeneSets <- read.csv('../1.Data/BindeaImmuneCells_GeneSets.txt',
                            sep='\t',
                            header=T)

In [9]:
head(Bindea_GeneSets)

activated_Dendritic_cells,B_cells,CD8_T_cells,Citotoxic_cells,Dendritic_cells,Eosinophils,immature_Dendritic_cells,Macrophages,Mast_cells,Neutrophils,⋯,T_central_memory_cells,T_effector_memory_cells,T_follicular_helper_cells,T_gamma_delta_cells,Th1_cells,Th17_cells,Th2_cells,Treg_cells,Angiogenesis,Antigen_presentation_machinery
CCL1,ABCB4,ABT1,APBA2,CCL13,ABHD2,ABCG2,APOE,ABCC4,ALPL,⋯,AQP3,AKT3,B3GAT1,C1orf61,APBB2,IL17A,ADCY1,FOXP3,CDH5,HLA-A
EBI3,BACH2,AES,APOL3,CCL17,ACACB,BLVRB,ATG7,ADCYAP1,BST1,⋯,ATF7IP,C7orf54,BLR1,CD160,APOD,IL17RA,AHI1,,ELTD1,HLA-B
INDO,BCL11A,APBA2,CTSW,CCL22,C9orf156,CARD9,BCAT1,CALB2,CD93,⋯,ATM,CCR2,C18orf1,FEZ1,ATP9A,RORC,ANK1,,CLEC14A,HLA-C
LAMP3,BLK,ARHGAP8,DUSP2,CD209,CAT,CD1A,CCL7,CEACAM8,CEACAM3,⋯,CASP8,DDX17,CDK5R1,TARP,BST2,,BIRC5,,LDB2,B2M
OAS3,BLNK,C12orf47,GNLY,HSD11B1,CCR3,CD1B,CD163,CMA1,CREB5,⋯,CDC14A,EWSR1,CHGB,TRD,BTG3,,CDC25C,,ECSCR,TAP1
,CCR9,C19orf6,GZMA,NPR1,CLC,CD1C,CD68,CPA3,CRISPLD2,⋯,CEP68,FLI1,CHI3L2,TRGV9,CCL4,,CDC7,,MYCT1,TAP2


In [10]:
Bindea_GeneSets <- convertRowsToList(t(Bindea_GeneSets))

## Run ssGSEA

In [11]:
bin_NES <- gsva(data_mtx,
                Bindea_GeneSets,
                method='ssgsea',
                min.sz=0,
                max.sz=1000,
                ssgsea.norm=T)

Estimating ssGSEA scores for 26 gene sets.
  |                                                                      |   0%Using parallel with 4 cores
  |======================================================================| 100%


In [12]:
bin_NES <- data.frame("Term"=rownames(bin_NES),
                      bin_NES)

rownames(bin_NES) <- NULL

In [15]:
head(bin_NES)

Term,A01,A02,A03,A04,A05,A06,A07,A08,A09,⋯,C06,C07,C08,C09,C10,C11,C12,D01,D02,D03
activated_Dendritic_cells,-0.066754509,0.097384283,-0.072684444,0.08406037,-0.039156324,-0.03913688,0.03107886,0.047359206,0.009225672,⋯,-0.138728449,-0.11743575,-0.09688468,-0.006450510,-0.11900309,-0.11655420,-0.173622562,-0.139019418,-0.11683955,-0.12814512
B_cells,-0.003080459,0.018623922,0.022715570,0.02951553,-0.010551928,-0.05771865,0.01858166,-0.033956971,-0.073086897,⋯,-0.047671127,0.00381040,-0.04950963,-0.002100965,0.02356159,0.10275271,0.051442694,-0.005627768,0.02556495,-0.03113372
CD8_T_cells,0.095100512,0.099430527,0.094088306,0.07429195,0.093516971,0.08977499,0.05386297,0.059920747,0.082284801,⋯,0.081629660,0.09933864,0.10938170,0.084270861,0.06676323,0.09050137,0.104727427,0.091541657,0.10927883,0.09812421
Citotoxic_cells,0.103809255,0.112686393,0.084147911,0.02977081,0.037928174,-0.02408805,-0.06285071,-0.041938113,-0.103344902,⋯,-0.066424407,-0.02344524,-0.03938905,-0.053673465,-0.04709108,0.05630751,0.008552687,-0.007122433,-0.01273871,-0.06110341
Dendritic_cells,-0.176476428,-0.136052028,-0.175691922,-0.14513787,-0.110276254,-0.11966672,-0.14789081,-0.109404501,-0.114896146,⋯,-0.102047487,-0.01734816,0.01414666,-0.065368738,-0.12973470,-0.03258076,-0.057819789,-0.085136366,-0.08762304,-0.05003171
Eosinophils,-0.034222013,0.007538603,-0.007610453,0.02250445,-0.002669838,-0.01267269,0.01926266,-0.004099934,-0.006889786,⋯,0.002467249,0.03968204,0.04808722,0.014791031,0.01835545,0.01526813,0.008219769,-0.016815433,-0.04023631,-0.03714241


### Save NES

In [16]:
write.table(bin_NES,
            file='../../Data/3/BindeaImmuneCells_TreatmentNaive_NES.txt',
            sep='\t',
            row.names=F,
            col.names=T)

# End script